In [1]:
import socket
import pickle
from API import get_pregunta_aleatoria, check_respuesta

socket_servidor = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
socket_servidor.bind(("127.0.0.1", 6969))

while True:
    bytes_rx, address = socket_servidor.recvfrom(1024)
    received_message = pickle.loads(bytes_rx)

    message = received_message[0]
    number_client = received_message[1]

    if message == "Ready to play":
        # Obtenemos una pregunta aleatoria y sus posibles respuestas
        pregunta_info = get_pregunta_aleatoria()
        pregunta_texto = pregunta_info["Pregunta"]
        posibles_respuestas = pregunta_info["Posibles Respuestas"]
        respuesta_correcta = pregunta_info["Respuesta Correcta"]

        # Enviamos la pregunta y las posibles respuestas al cliente
        socket_servidor.sendto(pickle.dumps((pregunta_texto, posibles_respuestas)), address)

        # Recibimos la respuesta del cliente
        bytes_rx, address = socket_servidor.recvfrom(1024)
        received_message = pickle.loads(bytes_rx)
        respuesta_cliente = received_message[0]

        # Validamos la respuesta y enviamos el resultado al cliente
        if check_respuesta(pregunta_texto, respuesta_cliente):
            resultado = "Server: Correct answer. Congratulations! \n"
        else:
            resultado = "Server: Incorrect answer. The correct answer is " + respuesta_correcta + ". \n"

        socket_servidor.sendto(pickle.dumps(resultado), address)
    if message == "Exit":
        print("Server: Client " + str(number_client) + " has left the game.")

socket_servidor.close()

